<a href="https://colab.research.google.com/github/dlsyscourse/lecture13/blob/main/13_hardware_acceleration_architecture_overview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture 13: Hardware Acceleration Implementation

In this lecture, we will to walk through backend scafoldings to get us hardware accelerations for needle.




## Select a GPU runtime type
In this lecture, we are going to make use of c++ and CUDA to build accelerated linear algebra libraries. In order to do so, please make sure you select a runtime type with GPU and rerun the cells if needed:
- Click on the "Runtime" tab
- Click "Change runtime type"
- Select GPU

After you started the right runtime, you can run the following command to check if there is a GPU available.

In [1]:
!nvidia-smi

Tue May  6 15:13:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.51.03              Driver Version: 576.28         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...    On  |   00000000:01:00.0  On |                  N/A |
| N/A   57C    P5             10W /   60W |    1574MiB /   4096MiB |     28%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Prepare the codebase

To get started, we can clone the related repo from the github.

In [ ]:
# Code to set up the assignment
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/
# !mkdir -p 10714f24
# %cd /content/drive/MyDrive/10714f24
# # comment out the following line if you run it for the second time
# # as you already have a local copy of lecture13
# #!git clone https://github.com/dlsyscourse/lecture13
# !rm -rf /content/needle
# !ln -s /content/drive/MyDrive/10714f24/lecture13 /content/needle

Mounted at /content/drive
/content/drive/MyDrive
/content/drive/MyDrive/10714f24
Cloning into 'lecture14'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 99 (delta 31), reused 85 (delta 23), pack-reused 0 (from 0)
Receiving objects: 100% (99/99), 53.49 KiB | 1.14 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [ ]:
# !python3 -m pip install pybind11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 4.2 MB/s eta 0:00:00


### Build the needle cuda library

We leverage pybind to build a c++/cuda library for acceleration. You can type make to build the corresponding library.

In [ ]:
%cd /content/needle
!make clean
!make

/content/drive/MyDrive/10714f24/lecture14
rm -rf build python/needle/backend_ndarray/ndarray_backend*.so
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Python: /usr/local/bin/python (found version "3.10.12") found components: 

We can then run the following command to make the path to the package available in colab's environment as well as the PYTHONPATH.

In [ ]:
%set_env PYTHONPATH /content/needle/python:/env/python
import sys
sys.path.append("/content/needle/python")

env: PYTHONPATH=/content/needle/python:/env/python


## Codebase walkthrough


Now click the files panel on the left side. You should be able to see these files

Python:
- needle/backend_ndarray/ndarray.py
- needle/backend_ndarray/ndarray_backend_numpy.py

C++/CUDA
- src/ndarray_backend_cpu.cc
- src/ndarray_backend_cuda.cu

The main goal of this lecture is to create an accelerated ndarray library.
As a result, we do not need to deal with needle.Tensor for now and will focus on backend_ndarray's implementation.

After we build up this array library, we can then use it to power backend array computations in needle.


## Creating a CUDA NDArray






In [ ]:
from needle import backend_ndarray as nd

Using needle backend


We can create a CUDA tensor from the data by specifying a device keyword.

In [ ]:
x = nd.NDArray([1, 2, 3], device=nd.cuda())

In [ ]:
y = x + 1

In [ ]:
x.numpy()

array([1., 2., 3.], dtype=float32)

In [ ]:
x.device

cuda()

In [ ]:
y = x + 1

In [ ]:
y.device

cuda()

In [ ]:
y.numpy()

array([2., 3., 4.], dtype=float32)

### Key Data Structures

Key data structures in backend_ndarray

- NDArray: the container to hold device specific ndarray
- BackendDevice: backend device
    - mod holds the module implementation that implements all functions
    - checkout ndarray_backend_numpy.py for a python-side reference.



## Trace GPU execution

Now, let us take a look at what happens when we execute the following code


In [ ]:
x = nd.NDArray([1, 2, 3], device=nd.cuda())
y = x + 1

In [ ]:
x.device.from_numpy

<function needle.backend_ndarray.ndarray_backend_cuda.PyCapsule.from_numpy>

In [ ]:
x = nd.NDArray([1, 2, 3])

In [ ]:
x.device.from_numpy

<function needle.backend_ndarray.ndarray_backend_numpy.from_numpy(a, out)>

Have the following trace:

backend_ndarray/ndarray.py
- `NDArray.__add__`
- `NDArray.ewise_or_scalar`
- `ndarray_backend_cpu.cc:ScalarAdd`

In [ ]:
y.numpy()

array([2., 3., 4.], dtype=float32)

Have the following trace:

- `NDArray.numpy`
- `ndarray_backend_cpu.cc:to_numpy`

## Guidelines for Reading C++/CUDA related Files

Read
- src/ndarray_backend_cpu.cc
- src/ndarray_backend_cuda.cu


Optional
- CMakeLists.txt: this is used to setup the build and likely you do not need to tweak it.







## NDArray Data Structure

Open up `python/needle/backend_ndarray/ndarray.py`.

An NDArray contains the following fields:
- handle: The backend handle that build a flat array which stores the data.
- shape: The shape of the NDArray
- strides: The strides that shows how do we access multi-dimensional elements
- offset: The offset of the first element.
- device: The backend device that backs the computation






## Transformation as Strided Computation

We can leverage the strides and offset to perform transform/slicing with zero copy.

- Broadcast: insert strides that equals 0
- Tranpose: swap the strides
- Slice: change the offset and shape

For most of the computations, however, we will call `array.compact()` first to get a contiguous and aligned memory before running the computation.

In [ ]:
import numpy as np


In [ ]:
x = nd.NDArray([1, 2, 3, 4], device=nd.cpu_numpy())

In [ ]:
x.numpy()

array([1., 2., 3., 4.], dtype=float32)

We can use strides and shape manipulation to create different views of the same array.

In [ ]:
y = nd.NDArray.make(shape=(2, 2), strides=(2, 1), device=x.device, handle=x._handle, offset=0)

In [ ]:
y.numpy()

array([[1., 2.],
       [3., 4.]], dtype=float32)

In [ ]:
z = nd.NDArray.make(shape=(2, 1), strides=(2, 1), device=x.device, handle=x._handle, offset=1)

In [ ]:
z.numpy()

array([[2.],
       [4.]], dtype=float32)

## CUDA Acceleration

Now let us open `src/ndarray_cuda_backend.cu` and take a look at current implementation of GPU ops.


## Steps for adding a new operator implementation
- Add an implementation in `ndarray_backend_cuda.cu`, expose via pybind
- Call into the operator in ndarray.py
- Write up testcases

In [ ]:
!make

CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- Found pybind11: /usr/local/lib/python3.10/dist-packages/pybind11/include (found version "2.13.6")
CMake Warning (dev) at CMakeLists.txt:55 (find_package):
  Policy CMP0146 is not set: The FindCUDA module is removed.  Run "cmake
  --help-policy CMP0146" for policy details.  Use the cmake_policy command to
  set the policy and suppress this warning.

This warning is for project developers.  Use -Wno-dev to suppress it.

-- Found cuda, building cuda backend
Tue Oct  8 01:33:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|------------------

If we directly run the code block, we will see an error, because ewise mul is not yet implemented

In [ ]:
x = nd.NDArray([1,2,3], device=nd.cuda())
x * 2

AttributeError: module 'needle.backend_ndarray.ndarray_backend_cuda' has no attribute 'ewise_mul'

## Connect back to needle Tensor

So far we only played with the `backend_ndarray` subpackage, which is a self-contained ndarray implementation within needle.

We can connect the ndarray back to needle as a backend.

In [ ]:
import needle as ndl

In [ ]:
x = ndl.Tensor([1,2,3], device=nd.cuda(), dtype="float32")
y = ndl.Tensor([2,3,5], device=nd.cuda(), dtype="float32")
z = x + y
z

needle.Tensor([3. 5. 8.])

In [ ]:
z.device

cuda()

In [ ]:
type(z.cached_data)

needle.backend_ndarray.ndarray.NDArray

## Write Standalone Python Test Files

Now that we have additional c++/cuda libraries in needle, we will need to type make in order to rebuild the library. Additionally, because the colab environment caches the old library, it is inconvenient to use the ipython cells to debug the updated library.




In [ ]:
!make

CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- Found pybind11: /usr/local/lib/python3.10/dist-packages/pybind11/include (found version "2.13.6")
CMake Warning (dev) at CMakeLists.txt:55 (find_package):
  Policy CMP0146 is not set: The FindCUDA module is removed.  Run "cmake
  --help-policy CMP0146" for policy details.  Use the cmake_policy command to
  set the policy and suppress this warning.

This warning is for project developers.  Use -Wno-dev to suppress it.

-- Found cuda, building cuda backend
Tue Oct  8 01:33:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|------------------


We recommend writing separate python files and invoke them from the command line. Create a new file `tests/mytest.py` and write your local tests. This is also a common develop practice in big projects that involves python c++ FFI.

In [ ]:
!python tests/mytest.py

python3: can't open file '/content/drive/MyDrive/10714f24/lecture14/tests/mytest.py': [Errno 2] No such file or directory


After we have building the library, we could choose to fully restart the runtime (factory reset runtime) if you want to bring the updated change back to another colab. Note that you will need to save your code changes to the drive or a private github repo.